<div >
<img src = "figs/ans_banner_1920x200.png" />
</div>

# Semana 5. Sesión Sincrónica.

El  *cuaderno* tiene como objetivo hacer una introducción a los sistemas de recomendación.

**NO** es necesario editar el archivo o hacer una entrega. Los ejemplos contienen celdas con código ejecutable (`en gris`), que podrá modificar libremente. Esta puede ser una buena forma de aprender nuevas funcionalidades del *cuaderno*, o experimentar variaciones en los códigos de ejemplo.



# Sistemas de Recomendación

## ¿Qué son los sistemas de recomendaciones?


Las preferencias de los individuos suelen seguir patrones que los sistemas de recomendación pueden aprovechar, por ejemplo,

- Si te interesó:  <div style="max-width:200px">
<img src = "figs/iron_man.jpg" />
</div>


- También te puede interesar: <div style="max-width:200px">
<img src = "figs/thor.jpg" />
</div> 




- Los sistemas de recomendación entonces encuentran patrones que son utilizados para predecir qué otros productos podrían gustarnos y generar sugerencias, de forma tal que  los usuarios encuentren contenido atractivo en un gran corpus. 

- Estos sistemas son muy exitosos, por ejemplo según un estudio del 2013 de [McKinsey](https://www.mckinsey.com/industries/retail/our-insights/how-retailers-can-keep-up-with-consumers), el 35% de los artículos comprados en Amazon surgen de estos sistemas de recomendación, y por lo tanto vale la pena estudiarlos cuidadosamente.

# Filtrado Colaborativo Basado en Items: Análisis de Canasta de Compra. 


## ¿Qué es el algoritmo A Priori?

El algoritmo A Priori se usa para encontrar patrones frecuentes en conjuntos de datos, especialmente para descubrir qué elementos aparecen juntos con frecuencia. Es muy usado en análisis de cestas de compra.

### Principio clave del A Priori

**Si un conjunto no es frecuente, ninguno de sus superconjuntos puede ser frecuente.** Esto permite podar eficientemente el espacio de búsqueda.

## Ejemplo práctico: Supermercado

Imaginemos que tenemos estas 5 transacciones de un supermercado:

**Transacciones:**
1. {Pan, Leche, Huevos}
2. {Pan, Mantequilla}
3. {Pan, Leche, Mantequilla, Huevos}
4. {Leche, Mantequilla}
5. {Pan, Leche}

## Ejemplo en Python

In [ ]:
import pandas as pd

df = pd.read_csv('data/Groceries_dataset.csv')
df.head()

In [ ]:
df['single_transaction'] = df['Member_number'].astype(str)+'_'+df['Date'].astype(str)

df.head()

In [ ]:
df2 = pd.crosstab(df['single_transaction'], df['itemDescription'])

# El último paso de preprocesamiento de datos consiste en codificar todos los valores del data frame anterior a booleanos (True/False). Esto significa que, aunque haya múltiples unidades del mismo artículo en una misma transacción, el valor se codificará como True, ya que el análisis de canasta de mercado no toma en cuenta la frecuencia de compra.
basket_input = df2.applymap(lambda x: x > 0)

basket_input.head()

In [ ]:
!pip install mlxtend

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(basket_input, min_support=0.001, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift")

rules.head()

# Filtrado colaborativo basado en embeddings


Estudiemos un ejemplo en el que queremos generar recomendaciones de películas a partir de *embeddings*. Supondremos que nuestra plataforma de streaming: **"SlowFlow"** tiene 4 usuarios y 5 películas: [Batman: El caballero de la noche asciende](https://es.wikipedia.org/wiki/The_Dark_Knight_Rises), [Harry Potter y la Piedra Filosofal](https://es.wikipedia.org/wiki/Harry_Potter_y_la_piedra_filosofal), [Shrek](https://es.wikipedia.org/wiki/Shrek), [Las trillizas de Belleville](https://es.wikipedia.org/wiki/Les_Triplettes_de_Belleville) y [Memento](https://es.wikipedia.org/wiki/Memento). 

La siguiente matriz que llamaremos $A$, donde $A\in\mathbb{R}^{ m\times n}$, es decir, las filas son los $m$ usuarios y las columnas las $n$ películas, muestra con un $\checkmark$ las películas que cada usuario vió:


<center>
<img src = "figs/colab0.png" alt = "embedding1D" style = "width: 500px;"/>
</center>


#### Embedding en una sola dimensión 

Comencemos representando las películas y los usuarios a un *embedding space*  de una sola dimensión. Por ejemplo, a lo largo del segmento $[-1,1]$ de forma tal que películas e individuos similares estén más cerca.



<center>
<img src = "figs/colab1.png" alt = "embedding1D" style = "width: 500px;"/>
</center>

<center>
<img src = "figs/colab2b.png" alt = "embedding1D" style = "width: 500px;"/>
</center>

#### Embeddings en dos dimensiones

<center>
<img src = "figs/colab4.png" alt = "embedding2D" style = "width: 500px;"/>
</center>

<center>
<img src = "figs/colab5.png" alt = "embedding2D" style = "width: 500px;"/>
</center>


Finalmente

<center>
<img src = "figs/colab3b.png" alt = "embedding2D" style = "width: 500px;"/>
</center>


#### Como obtenemos los embeddings?

Una forma de obtener embeddings es mediante la Descomposición en Valores Singulares (SVD):

$$ A = U\Sigma V' $$

En el contexto tradicional de SVD, 
    
 - $U$ es una matriz ortogonal cuyas columnas son los vectores singulares de la matriz original $A$, que están relacionados con los **vectores propios (característicos)** de $AA'$. Estos vectores también tienen una estrecha relación con las **componentes principales** en Análisis de Componentes Principales (PCA), ya que representan las direcciones principales de variabilidad en los datos para las filas de $A$. 
 - Por otro lado, $V$ es otra matriz ortogonal cuyas columnas son los vectores singulares de $A$, pero asociados con las columnas de la matriz original, representando las direcciones principales de variabilidad en las columnas de $A$. En otras palabras, $U$ y $V$ son matrices que describen cómo las filas y las columnas de $A$ se pueden proyectar en espacios de características latentes.

En el **contexto de embeddings**, $U$ y $V$ tienen una interpretación más específica.
    
 - $U$ representa la matriz de **embeddings de los usuarios**, es decir, cada fila de $U$ es una representación latente de un usuario en un espacio de características, capturando sus preferencias a lo largo de diferentes dimensiones latentes. 
 - Similarmente, $V$ es la matriz de **embeddings de los ítems**, donde cada columna de $V$ describe un ítem en ese mismo espacio latente, permitiendo modelar cómo los ítems están relacionados con esas características latentes identificadas.

Este **espacio de características latentes** es comparable a lo que ocurre en el contexto de reconocimiento facial con las **Eigenfaces**. En Eigenfaces, las imágenes de rostros se proyectan en un espacio de características latentes donde cada dimensión captura un patrón característico de los rostros, como la forma general o las características distintivas. De manera similar, en el caso de embeddings obtenidos a partir de SVD, tanto los usuarios como los ítems se proyectan en un espacio de características latentes que captura los patrones más representativos de sus interacciones, como preferencias o similitudes. En ambos casos, el objetivo es reducir la dimensionalidad mientras se preserva la información más relevante.

La matriz $\Sigma$ sigue conteniendo los valores singulares, que actúan como ponderadores, indicando qué tan importantes son estas características latentes para describir las interacciones entre usuarios e ítems.

Para reducir la dimensionalidad y obtener una representación de rango inferior, seleccionamos solamente las primeras $k$ características de estas matrices, que consideramos capturan de manera más efectiva las preferencias y los gustos de los usuarios.

####  Interpretación de Embeddings

##### Ejemplo 1:

In [ ]:
import pandas as pd
import numpy as np

# Cargamos y visualizamos  los datos
pelis = pd.read_csv('data/pelis.csv')
pelis = pelis.set_index("Usuarios")
pelis

In [ ]:
np.linalg.matrix_rank(pelis.to_numpy())

In [ ]:
U, S, Vt = np.linalg.svd(pelis, full_matrices=False)

In [ ]:
print(U[:,0:2])

In [ ]:
print(Vt[0:2,:])

In [ ]:
n=2
k=2
S2= np.resize(S,[n,1])*np.eye(n,k) #ponemos los valores singulares en una matriz diagonal

print(S2)

In [ ]:
#Reconstrucción
l=2
pd.DataFrame(np.dot(U[:,0:l],np.dot(S2[0:l,0:l],Vt[0:l,:]))).round()

##### Ejemplo 2:

In [ ]:
# Cargamos y visualizamos  los datos
pelis2 = pd.read_csv('data/pelis2.csv')
pelis2 = pelis2.set_index("Usuarios")
pelis2

In [ ]:
np.linalg.matrix_rank(pelis2.to_numpy())

In [ ]:
U, S, Vt = np.linalg.svd(pelis2,full_matrices=False)
print('U, S, Vt ='), U, S, Vt

In [ ]:
n=3
k=3
S2= np.resize(S,[n,1])*np.eye(n,k) #ponemos los valores singulares en una matriz diagonal
l=3
pd.DataFrame(np.dot(U[:,0:l],np.dot(S2[0:l,0:l],Vt[0:l,:]))).round()

# Evaluación Sistemas de Recomendación


- La evaluación de los sistemas de recomendación es otro paso importante para evaluar la efectividad del método. 

- En la literatura podemos ver dos tipologías principales: evaluación offline y online.


##  Offline
 - La evaluación offline es la que estamos acostumbrados en ML supervisado donde usamos un conjunto de entrenamiento y un conjunto de prueba; y aplicamos la métrica de evaluación apropiada. 
    Por ejemplo si se trata de etiquetas numéricas, como las calificaciones de 5 estrellas, la forma más común de validar un sistema de recomendación se basa en su valor de predicción, es decir, la capacidad de predecir las calificaciones del usuario. Las funciones estándar como el error cuadrático medio (RMSE), la precisión, las curvas ROC, etc...


##  Online    
 
 - La evaluación online es cuando utilizamos un conjunto de herramientas que nos permite observar las interacciones de los usuarios con el sistema. 
 - La técnica en línea más común se llama prueba A-B y tiene la ventaja de permitir la evaluación del sistema al mismo tiempo que los usuarios aprenden, compran o juegan con el sistema de recomendación. 

<div style="max-width:500px">
<img src = "figs/AB.jpg" />
</div>

    
 - Esto acerca la evaluación al funcionamiento real del sistema y la hace realmente efectiva cuando el propósito del sistema es cambiar o influir en el comportamiento de los usuarios. 
 - Para evaluar la prueba, estamos interesados en medir cómo cambia el comportamiento del usuario cuando el usuario interactúa con diferentes sistemas de recomendación. Y require estimación de parámetros causales.

<div style="max-width:500px">
<img src = "figs/BING.png" />
</div>



    
- En esta situación, las métricas fuera de línea como RMSE no son lo suficientemente buenas. En este caso, estamos particularmente interesados en la evaluación del objetivo global del sistema de recomendación, la retención de usuario.
